# Introduction to Data Science – Homework 6
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, March 01 2024, 11:59pm.

In Part 1 of this homework you will scrape github repositories and organize the information in a Pandas dataframe. In Part 2, you will use linear regression to gain meaningful insights. 

## Your Data
First Name: Logan
<br>
Last Name: Correa
<br>
E-mail: u1094034@umail.utah.edu
<br>
UID: u1094034
<br>

In [3]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import time
import os

import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 
# where the data is stored
DATA_PATH = "data"

### 1. Scrape Github Repository List using BeautifulSoup
In this part you will explore Github repositories, specifically the 100 most-starred repositories. You are going to scrape data from a snapshot of [this repository list](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories).

### 1.1. Check whether you are permitted to scrape the data
Before you start to scrape any website you should go through the terms of service and policy documents of the website. Almost all websites post conditions to use their data. Check the terms of [https://github.com/](https://github.com/) (see the tiny "terms" link at the bottom of the page) to see whether the site permits you to scrape their data or not. Are you sure you are allowed to scrape?

**Your solution:**

Scraping is allowed for researchers as long as published works are open access and the information is not used for spamming purposes.

### Task 1.2 Load the Data

To avoid any problems with GitHub blocking us from downloading the data many times, we have downloaded and saved a snapshot of the html files for you in the [data](data) folder. Note that the data folder is not completely consistent with what you see on the web – we've made a few patches to the data that makes your task here easier and this data represents a snapshot in time. You will be treating the data folder as your website to be scraped. The path to data folder is stored in `DATA_PATH` variable.

In the data folder you will find first 10 pages of highly starred repositories saved as `searchPage1.html`,`searchPage2.html`,`searchPage3.html` ... `searchPage10.html`

Check out page 10 if you want to see what happens if you scrape too quickly 😉. 

Now read these html files in python and create a soup object. This is a two step process:
 * Read the text in the html files
 * Create the soup from the files that you've read. 

In [4]:
# Read html files and create soup files
html_pages = []
for files in os.listdir(DATA_PATH):
    if files.endswith(".html"):
        full_path = os.path.join(DATA_PATH, files)
        with open(full_path, 'r') as f:
            html_pages.append(f.read())

SearchPage_soup = []
for page in html_pages:
    SearchPage_soup.append(BeautifulSoup(page, 'html.parser'))

len(SearchPage_soup)

10

90
['netdata/netdata', 'tonsky/FiraCode', 'denoland/deno', 'h5bp/html5-boilerplate', 'ElemeFE/element', 'adam-p/markdown-here', 'h5bp/Front-end-Developer-Interview-Questions', 'resume/resume.github.com', 'josephmisiti/awesome-machine-learning', 'lodash/lodash', 'angular/angular.js', 'puppeteer/puppeteer', 'mrdoob/three.js', 'microsoft/TypeScript', 'angular/angular', 'microsoft/terminal', 'laravel/laravel', 'moby/moby', 'ant-design/ant-design', 'iluwatar/java-design-patterns', 'ossu/computer-science', '30-seconds/30-seconds-of-code', 'mui-org/material-ui', 'jquery/jquery', 'webpack/webpack', 'reduxjs/redux', 'nvbn/thefuck', 'vuejs/awesome-vue', 'avelino/awesome-go', 'atom/atom', 'apple/swift', 'hakimel/reveal.js', 'MisterBooo/LeetCodeAnimation', 'PanJiaChen/vue-element-admin', 'pallets/flask', 'socketio/socket.io', 'expressjs/express', 'Semantic-Org/Semantic-UI', 'shadowsocks/shadowsocks-windows', 'chartjs/Chart.js', 'jwasham/coding-interview-university', 'kamranahmedse/developer-roadma

### Extracting Data

Extract the following data for each repository, and create a Pandas Dataframe with a row for each repository and a column for each of these datums. 

+ The name of the repository
+ The primary language (there are multiple or none, if multiple, use the first one, if none, use "none")
+ The number of watches
+ The number of stars
+ The number of forks
+ The number of issues
+ Number of commits
+ Number of contributors
+ Number of pull requests, and
+ Number of top level folders in the file list.

Here's an example for one repository, `jackfrued/Python-100-Days,` in our dataset: 
```python
{'name': 'Python-100-Days',
'language': 'Jupyter Notebook',
'watches': '4822',
'stars': '78068',
'forks': '30979',
'issues': 224,
'commits': 296,
'contributors': 12,
'pull_requests':85,
'folders': 14
}
```

### Task 1.3 Extract repository URLs

If you look at the results of the 100 most-starred repositories [(this list)](https://github.com/search?o=desc&q=stars%3A%3E1&s=stars&type=Repositories), you will notice that all the information we want to extract for each repository is not in that list. This information is in the repository’s individual web page, for example [996icu](https://github.com/996icu/996.ICU). 

Therefore, you will first have to extract links of each repository from the soup you scraped earlier. When you extract the link for the repository, it will be a path to the stored HTML page for the repository. You will use this path to read the file and extract the above information.

Refer to the scraping lecture for details on how to do this. We recommend you use the web inspector to identify the relevant structures.

Example of a link that you need to extract - 996icu/996.ICU.html

In [57]:

# Extract all 'div' elements with class "mt-n1" (these classes contain individual repositories) from each item in SearchPage_soup.
repositories = []

for i in range(0, len(SearchPage_soup)):
    mtn1 = SearchPage_soup[i].find_all('div', class_="mt-n1")
    for item in mtn1:
        repo_links = item.find('a', class_="v-align-middle")
        repositories.extend(repo_links.text.split())

print(len(repositories))
print(repositories)

repositories = [repo_path + '.html' for repo_path in repositories]

htmls = []

for repo_path in repositories:
    # Construct the full path to the .html file
    full_file_path = os.path.join(DATA_PATH, repo_path)

    # Check if the file exists
    if os.path.isfile(full_file_path):
        # Open and read the file
        with open(full_file_path, 'r') as file:
            html_content = file.read()
            htmls.append(html_content)

print(len(htmls))




90
['netdata/netdata', 'tonsky/FiraCode', 'denoland/deno', 'h5bp/html5-boilerplate', 'ElemeFE/element', 'adam-p/markdown-here', 'h5bp/Front-end-Developer-Interview-Questions', 'resume/resume.github.com', 'josephmisiti/awesome-machine-learning', 'lodash/lodash', 'angular/angular.js', 'puppeteer/puppeteer', 'mrdoob/three.js', 'microsoft/TypeScript', 'angular/angular', 'microsoft/terminal', 'laravel/laravel', 'moby/moby', 'ant-design/ant-design', 'iluwatar/java-design-patterns', 'ossu/computer-science', '30-seconds/30-seconds-of-code', 'mui-org/material-ui', 'jquery/jquery', 'webpack/webpack', 'reduxjs/redux', 'nvbn/thefuck', 'vuejs/awesome-vue', 'avelino/awesome-go', 'atom/atom', 'apple/swift', 'hakimel/reveal.js', 'MisterBooo/LeetCodeAnimation', 'PanJiaChen/vue-element-admin', 'pallets/flask', 'socketio/socket.io', 'expressjs/express', 'Semantic-Org/Semantic-UI', 'shadowsocks/shadowsocks-windows', 'chartjs/Chart.js', 'jwasham/coding-interview-university', 'kamranahmedse/developer-roadma

In [ ]:
soups = []
for page in htmls:
    soups.append(BeautifulSoup(page, 'html.parser'))

print(len(soups))

In [6]:
htmls = []
for root, dirs, files in os.walk(DATA_PATH):
    for file in files:
        if file.endswith(".html") and not any(file == f"searchPage{i}.html" for i in range(1, 11)):
            full_path = os.path.join(root, file)
            with open(full_path, 'r') as f:
                htmls.append(f.read())

soups = []
for page in htmls:
    soups.append(BeautifulSoup(page, 'html.parser'))

print(len(soups))


90


In [7]:
# Repository Names
repo_names = []

for soup in soups:
    element = soup.find("a", attrs={"data-pjax": "#js-repo-pjax-container"})
    repo_names.append(element.text)

# Languages
lang = []

for soup in soups:
    element = soup.find("span", class_="language-color")
    if element:
        lang.append(element.text)
    else:
        lang.append("None")

print(len(lang))
print(lang)

# Number of watchers
watchers = []

for soup in soups:
    element = soup.find("a", class_="social-count", href=lambda x: x and "/watchers" in x)
    watchers.append(element.text.strip())

# Number of stars
stars = []

for soup in soups:
    element = soup.find('a', class_="social-count js-social-count", href=lambda x: x and "/stargazers" in x)
    stars.append(element.text.strip())

# Number of forks
forks = []

for soup in soups:
    soup.find('a', class_="social-count", href=lambda x: x and "/members" in x)
    forks.append(element.text.strip())

# Number of Issues
issues = []

for soup in soups:
    issue_link = soup.find('a', href=lambda x: x and '/issues' in x)
    if issue_link:
        issue_count = issue_link.find('span', class_='Counter')
        if issue_count:
            issues.append(issue_count.text.strip())
        else:
            issues.append("No Field Found")
    else:
        issues.append("No Field Found")

# Number of commits
commits = []

for soup in soups:
    commit_link = soup.find('a', href=lambda x: x and '/commits/master' in x)
    if commit_link:
        commit_count = commit_link.find('span', class_='num text-emphasized')
        if commit_count:
            commits.append(commit_count.text.strip())
        else:
            commits.append("No Field Found")
    else:
        commits.append("No Field Found")   

#  Number of contributors
contributors = []

for soup in soups:
    contributors_link = soup.find('a', href=lambda x: x and '/contributors' in x)
    if contributors_link:
        contributors_count = contributors_link.find('span', class_='num text-emphasized')
        if contributors_count:
            contributors.append(contributors_count.text.strip())
        else:
            contributors.append("No Field Found")
    else:
        contributors.append("No Field Found")

# Number of pulls
pulls = []

for soup in soups:
    pulls_link = soup.find('a', href=lambda x: x and '/pulls' in x)
    if pulls_link:
        pulls_count = pulls_link.find('span', class_='Counter')
        if pulls_count:
            pulls.append(pulls_count.text.strip())
        else:
            pulls.append("No Field Found")
    else:
        pulls.append("No Field Found")

90
['C', 'None', 'None', 'Go', 'None', 'Jupyter Notebook', 'Python', 'TypeScript', 'Python', 'Go', 'Python', 'TypeScript', 'JavaScript', 'None', 'Python', 'C', 'Python', 'Python', 'Vue', 'JavaScript', 'None', 'None', 'Java', 'JavaScript', 'TypeScript', 'Go', 'JavaScript', 'JavaScript', 'Ruby', 'JavaScript', 'Rust', 'None', 'JavaScript', 'HTML', 'JavaScript', 'Assembly', 'C++', 'C#', 'JavaScript', 'JavaScript', 'JavaScript', 'JavaScript', 'JavaScript', 'JavaScript', 'Clojure', 'JavaScript', 'JavaScript', 'None', 'Java', 'Python', 'Dart', 'TypeScript', 'None', 'JavaScript', 'JavaScript', 'JavaScript', 'JavaScript', 'C++', 'Go', 'Python', 'C++', 'JavaScript', 'Java', 'None', 'Java', 'TypeScript', 'Shell', 'Python', 'Java', 'JavaScript', 'Python', 'JavaScript', 'JavaScript', 'JavaScript', 'CSS', 'PHP', 'Vue', 'TypeScript', 'TypeScript', 'C++', 'Java', 'JavaScript', 'None', 'JavaScript', 'Python', 'JavaScript', 'JavaScript', 'JavaScript', 'JavaScript', 'Python']


In [8]:

# Number of folders?

In [9]:
data_dict = {"Name": repo_names, "Language": lang, "Watchers": watchers, "Stars": stars, "Forks": forks, "Issues": issues, "Commits": commits, "Contributors": contributors, "Pull_Requests": pulls}
project_info = pd.DataFrame(data_dict)

### Task 1.4 Extracting required information

Once you have extracted links for each repository, you can start parsing those HTML pages using BeautifulSoup and extract all the required information.

**Note**: There are few repositories which do not contain 'issues' field (such as 996icu/996.ICU.html). Therefore, write your code such that it handles this condition as well.

**Save the dataframe you created to a new file project_info.csv and include this in your submission.** This separate file will also be graded and is required to earn points.

You also need to make sure that you reformat all numerical columns to be integer data. You can do that either as you parse, or when you have a dataframe with strings.

Note that there is one repository flagged as having infinite contributers (the Linux kernel). We'll assume that it in fact has 15600 contributors (an estimate based on a Google search at the time of download).

In [10]:
def int_convert(value):
    value_str = str(value)
    if 'k' in value_str:
        # Remove 'k' and convert to float, then multiply by 1000
        return int(float(value.replace('k', '')) * 1000)
    else:
        return int(value)

# Apply the conversion function to the 'Counts' column
project_info['Watchers'] = project_info['Watchers'].apply(int_convert)
project_info['Stars'] = project_info['Stars'].apply(int_convert)
project_info['Forks'] = project_info['Forks'].apply(int_convert)


In [11]:
display(project_info)

,Name,Language,Watchers,Stars,Forks,Issues,Commits,Contributors,Pull_Requests
0,netdata,C,1400,44800,82700,680,"9,816",323,37
1,computer-science,None,3400,54900,82700,8,No Field Found,91,1
2,the-art-of-command-line,None,2200,68200,82700,78,"1,185",150,89
3,awesome-go,Go,2600,51900,82700,8,"3,328","1,195",19
4,free-programming-books,None,8600,137000,82700,19,"5,200","1,096",16
...,...,...,...,...,...,...,...,...,...
85,bootstrap,JavaScript,7200,139000,82700,331,"19,337","1,108",78
86,create-react-app,JavaScript,1800,76000,82700,430,"2,493",787,108
87,react-native,JavaScript,3700,84900,82700,721,"19,379","2,062",84
88,react,JavaScript,6600,144000,82700,483,"12,812","1,361",98


In [12]:


project_info.to_csv('project_info.csv', index=False)

### 2. Analyzing the repository data

In this part, you will analyze the data collected in Part 1 using regression tools. The goal is to identify properties that make a repository popular. 

First, load the `project_info.csv` file in again. **We need you to do this so that we can run your code below without having to run your scraping code, which can be slow.**

In [13]:
project_info = pd.read_csv('project_info.csv')
project_info.head()

,Name,Language,Watchers,Stars,Forks,Issues,Commits,Contributors,Pull_Requests
0,netdata,C,1400,44800,82700,680,"9,816",323,37
1,computer-science,NaN,3400,54900,82700,8,No Field Found,91,1
2,the-art-of-command-line,NaN,2200,68200,82700,78,"1,185",150,89
3,awesome-go,Go,2600,51900,82700,8,"3,328","1,195",19
4,free-programming-books,NaN,8600,137000,82700,19,"5,200","1,096",16


### Task 2.1 Describe the data

+ Get an overview of the data using the describe function.
+ Compute the correlation matrix, visualize it with a heat map.
+ Visualize the correlations by making a scatterplot matrix.
+ Interprete what you see.

You can re-use code from your previous homework here.

In [14]:
# your code goes here

**Your Interpretation:** TODO

### 2.2. Linear regression

1. Use linear regression to try to predict the number of Stars based on Forks, Pull Requests, and Number of Folders. Explain why this is not a very good model by discussing the R-squared , F-statistic p-value, and coefficient  p-values. 
+ Develop another model which is better. Explain why it is better and interpret your results. Hint: try using other variables such as Watches and/or Contributors. 

In [15]:
# your code goes here

**Your interpretation:** TODO